# ipyleaflet: Interactive maps

## A Jupyter - LeafletJS bridge

## https://github.com/jupyter-widgets/ipyleaflet


ipyleaflet is a jupyter interactive widget library which provides interactive maps to the Jupyter notebook.

- MIT Licensed

**Installation:**

```bash
conda install -c conda-forge ipyleaflet
```

In [ ]:
from __future__ import print_function

from ipyleaflet import (
    Map,
    Marker, MarkerCluster, TileLayer, ImageOverlay, GeoJSON,
    Polyline, Polygon, Rectangle, Circle, CircleMarker, Popup,
    SplitMapControl, WidgetControl,
    basemaps, basemap_to_tiles
)

from ipywidgets import HTML

In [ ]:
center = [34.6252978589571, -77.34580993652344]
zoom = 10

# Map Widget

In [ ]:
m = Map(center=center, zoom=zoom)
m

In [ ]:
m.layout.height = '700px'

### Every Layer in ipyleaflet is a Widget: TileLayers, Markers, Heatmaps, Videos...

You can dynamically change the layers you apply on top of the Map, for example you can dynamically change the base TileLayer and control it through a SelectionSlider widget:

In [ ]:
from datetime import date, timedelta
from ipywidgets import SelectionSlider

n_days = 30
days = []

# Previous 30 days, starting from today
for n in range(n_days):
    delta = (n_days - n) * timedelta(days=1)
    days.append((date.today() - delta).isoformat())

days_slider = SelectionSlider(options=days, value=days[-1], description='Day: ')

base_layer = basemap_to_tiles(basemaps.NASAGIBS.ModisTerraTrueColorCR, days_slider.value)

m.layers = [base_layer]
m.zoom = 6

In [ ]:
def handle_day_change(change, **kwargs):
    base_layer.url = basemaps.NASAGIBS.ModisTerraTrueColorCR.get('url') % change['new']

days_slider.observe(handle_day_change, 'value')

days_slider

# jupyterlab-sidecar: A sidecar output widget for JupyterLab

## https://github.com/jupyter-widgets/jupyterlab-sidecar

In [ ]:
from sidecar import Sidecar

sc = Sidecar(title='Map Output')
with sc:
    m.layout.height = ''
    display(m)

## Marker

In [ ]:
mark = Marker(location=m.center)

In [ ]:
m += mark

In [ ]:
mark.visible

In [ ]:
mark.visible = False

In [ ]:
mark.visible = True

In [ ]:
mark.interact(opacity=(0.0, 1.0, 0.01))

## Popup

The popup is displayed when you click on the marker. You can add ANY widget as a popup for the marker, from simple HTMLWidget to plots using bqplot.

In [ ]:
from ipywidgets import FloatSlider, link

slider = FloatSlider(value=1.0, min=0.0, max=1.0)
link((mark, 'opacity'), (slider, 'value'))

mark.popup = slider

## Marker Cluster

Markers can be clustered depending on the zoom level.

In [ ]:
xscale = 5
yscale = 10

x = [m.center[0] + i * xscale * .05 for i in (-1,0,1)]
y = [m.center[1] + i * yscale * .05 for i in (-1,0,1)]

from itertools import product
locations = product(x, y)
markers = [Marker(location=loc) for loc in locations]

The `MarkerCluster` will automatically handle clustering and the zoom level changes.

In [ ]:
marker_cluster = MarkerCluster(markers=markers)
m += marker_cluster

## Heatmap

In [ ]:
from ipywidgets import Button, IntSlider, link
from ipyleaflet import Heatmap
from random import gauss
import time

In [ ]:
center = (37.09, -103.66)
zoom = 5

In [ ]:
def create_random_data(length):
    "Return a list of some random lat/lon/value triples."
    return [[gauss(center[0], 2), 
             gauss(center[1], 4),
             gauss(700, 300)] for i in range(length)]

In [ ]:
m.center = center
m.zoom = zoom

In [ ]:
heat = Heatmap(locations=create_random_data(1000), radius=20, blur=10)
m.add_layer(heat)

In [ ]:
def generate(_):
    heat.locations = create_random_data(1000)

button = Button(description='Generate data', button_style='success')
button.on_click(generate)
button

In [ ]:
slider = IntSlider(min=10, max=30, value=heat.radius)
link((slider, 'value'), (heat, 'radius'))
slider

## Controls

In [ ]:
# The following NASA images need a zoom level <= 9
if m.zoom > 9:
    m.zoom = 9

control = SplitMapControl(
    left_layer=basemap_to_tiles(basemaps.NASAGIBS.ModisTerraTrueColorCR, "2017-11-11") , 
    right_layer=basemap_to_tiles(basemaps.NASAGIBS.ModisAquaBands721CR, "2017-11-11")
)
m.add_control(control)

In [ ]:
m.remove_control(control)

In [ ]:
from ipywidgets import IntSlider, Button, link

button = Button(description='Goto NYC')

def goto_nyc(*args, **kwargs):
    # NYC: 40.7128° N, 74.0060° W
    m.center = (40.7128, -74.0060)
    m.zoom = 9

button.on_click(goto_nyc)

wid_control = WidgetControl(widget=button, position='bottomright')
m.add_control(wid_control)

## JupyterLab Themes Support

Controls in ipyleaflet have support for JupyterLab themes, try changing the current theme

In [ ]:
m.remove_control(wid_control)

## Advanced example 1: Velocity

*It is ok to skip this for now -- running it requires downloading a ~1MB data file.*

This example needs an extra dependency that is not in the installation instructions and a dataset.

You can install it with this:

```bash
pip install netcdf4
```

To download the data, copy and paste the code below into a new cell and run it.

```python
import requests

wind_data = requests.get('https://github.com/benbovy/xvelmap/raw/master/notebooks/wind-global.nc')  

with open('wind-global.nc', 'wb') as f:  
    f.write(wind_data.content)
```


In [ ]:
from ipyleaflet.velocity import Velocity
import xarray as xr

In [ ]:
center = (0, 0)
zoom = 4

m2 = Map(center=center, zoom=zoom, interpolation='nearest', basemap=basemaps.CartoDB.DarkMatter)

sc2 = Sidecar(title='Map Velocity')

with sc2:
    display(m2)

In [ ]:
ds = xr.open_dataset('wind-global.nc')
display_options = {
    'velocityType': 'Global Wind',
    'displayPosition': 'bottomleft',
    'displayEmptyString': 'No wind data'
}
wind = Velocity(data=ds,
                zonal_speed='u_wind',
                meridional_speed='v_wind',
                latitude_dimension='lat',
                longitude_dimension='lon',
                velocity_scale=0.01,
                max_velocity=20,
                display_options=display_options)
m2.add_layer(wind)

# Advanced example

In [ ]:
from ipywidgets import Text, HTML, HBox
from ipyleaflet import GeoJSON, WidgetControl, Map 
import json

In [ ]:
m = Map(center = (43,-100), zoom = 4)

geo_json_data = json.load(open('us-states-density-colored.json'))
geojson = GeoJSON(data=geo_json_data, hover_style={'color': 'black', 'dashArray': '5, 5', 'weight': 2})
m.add_layer(geojson)

html = HTML('''
    <h4>US population density</h4>
    Hover over a state
''')
html.layout.margin = '0px 20px 20px 20px'
control = WidgetControl(widget=html, position='topright')
m.add_control(control)

def update_html(properties, **kwargs):
    html.value = '''
        <h4>US population density</h4>
        <h2><b>{}</b></h2>
        {} people / mi^2
    '''.format(properties['name'], properties['density'])

geojson.on_hover(update_html)

m

# Exercise

### 1) Create a Map

### 2) Create a slider controlling the zoom level of the Map (manually, not using interact)

### 3) Embed the slider in a WidgetControl on the bottom left of the map